# Ajuste de los parámetros del modelo IBM Granite para perfeccionar el resultado

Este proyecto utiliza un modelo de lenguaje de IBM Granite a través de la API de Replicate para analizar reseñas de productos.


## Objetivos

- Investigar cómo los distintos parámetros de generación (como top_k, top_p, max_tokens, etc.) afectan la calidad y precisión de las respuestas generadas por el modelo Granite.

- Evaluar la coherencia, creatividad o precisión de las salidas según diferentes configuraciones.

- Encontrar una combinación óptima de parámetros que se alinee mejor con el tipo de tarea (resumen, clasificación, extracción...).

- Aplicar principios de prompt engineering junto con ajustes de modelo para lograr resultados más controlados.

## Recursos necesarios:

In [ ]:
!pip install langchain_community
!pip install replicate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.9 MB/s eta 0:00:00


## Clasificación de datos

In [ ]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata

# Establecer el token de API
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token

# Configuración del modelo
model = "ibm-granite/granite-3.3-8b-instruct"

output = Replicate(
    model = model,
    replicate_api_token = api_token,
)

# Definir las reviews de los customers
customer_reviews = [
    "The battery lasts all day, bnut the phone gets hot during gaming.",
    "The screen is too dim outdoors, but I love the colors indoors.",
    "This phone is fast, but it keeps crashing when I open certain apps."

]

# Refinar la solicitud para incluir reseñas
reviews_text = "\n".join([f'Review {i+1}: {review}' for i, review in enumerate (customer_reviews)])

# Set model parameters for prompting with default values
parameters = {
    "top_k": 1,
    "top_p": 0.5,
    "max_tokens": 3,
    "min_tokens": 1,
    "random_seed": None,
    "repetition_penalty": 1.5,
    "stopping_criteria": "length",
    "stopping_sequence": " "
}

# Add initial prompt
refined_prompt = f"""
Classify these reviews as positive, negative, or mixed, and
tag relevant focus areas such as battery life, screen queality,
or performance

{reviews_text}
"""

# Invoke the model
response = output.invoke(refined_prompt, parameters=parameters)

# Print the response
print("Granite Model Refined Response:\n")

print(response)

Granite Model Refined Response:

1. Review 1: Negative (Battery life - Positive, Performance/Temperature - Negative)
The user expresses satisfaction with the battery life lasting all day, but also mentions a problem with the phone overheating during gaming, indicating a negative aspect related to performance under stress.

2. Review 2: Mixed (Screen Quality - Mixed)
This review is mixed. The user appreciates the color accuracy of the screen indoors but finds it too dim outdoors, highlighting issues with screen quality under different lighting conditions.

3. Review 3: Negative (Performance - Negative)
This review is negative as the user points out that despite the phone being fast, it crashes when opening certain apps, indicating a significant performance issue.


## Resumen de datos

In [ ]:
# Define the customer reviews
customer_meetings = [
    """
    La reunión comenzó con una discusión sobre el presupuesto de
    marketing del tercer trimestre. Se decidió que el 40 % del
    presupuesto se destinará a anuncios digitales, el 30 % a
    eventos y el 30 % a campañas en redes sociales. El equipo
    enfatizó la necesidad de concretar asociaciones con influencers
    para aumentar la visibilidad de la marca y el marketing por
    correo electrónico para impulsar la interacción directa. El
    próximo mes se lanzará un programa piloto para probar nuevos
    formatos de anuncios y el equipo revisará los resultados a
    finales del tercer trimestre.
    Posteriormente, el equipo discutió las métricas de rendimiento
    de la campaña. El seguimiento del ROI será una máxima prioridad
    y se realizarán ajustes en función de los datos de rendimiento.
    El equipo de eventos planteó inquietudes sobre la asignación de
    recursos para las próximas ferias comerciales y se acordó que
    se reasignarían $10,000 adicionales para cubrir estos gastos.
    Por último, el equipo revisó nuevos conceptos creativos para la
    próxima campaña y decidió continuar con el Concepto B, que tuvo
    mejores resultados entre los grupos focales. Se ultimaron los
    plazos para la entrega de los activos de campaña: todos los
    entregables deben presentarse a más tardar el 15 de julio.
    """
]

# Refine the prompt to include reviews
reviews_text = "\n".join([f"Review {i+1}:{review}" for i, review in enumerate(customer_meetings)])

# Apply default values for the parameters
parameters = {
    "top_k": 10,
    "top_p": 0.9,
    "max_tokens": 20,
    "min_tokens": 0,
    "random_seed": None,
    "repetition_penalty": 1.5,
    "stopping_criteria": "length",
    "stopping_sequence": " "
}

# Define the prompt with refined focus area prompt
refined_focusarea_prompt = f"""
Summarize this meeting by focusing on key points, decisions, and action times

{reviews_text}
"""

# Define the output
response = output.invoke(refined_focusarea_prompt, parameters=parameters)

# Print the response
print("Granite Model Response for refined focus area response:\n")
print(response)

Granite Model Response for refined focus area response:

**Meeting Summary:**

1. **Marketing Budget Allocation (Q3):**
   - 40% allocated to digital ads
   - 30% to events
   - 30% to social media campaigns
   - Focus on influencer partnerships and email marketing to boost brand visibility and direct engagement.

2. **Pilot Program for New Ad Formats:**
   - Launch pilot program next month
   - Review results by end of Q3

3. **Performance Metrics & ROI:**
   - Prioritize tracking Return on Investment (ROI)
   - Make adjustments based on performance data

4. **Event Resource Allocation:**
   - Additional $10,000 reallocated to cover upcoming trade shows, addressing team's concerns

5. **Creative Concepts for Next Campaign:**
   - Selected Concept B over other options due to better focus group results
   - Final deadlines set: All campaign assets to be submitted by July 15

**Next Review:** Performance data and pilot program results review at the end of Q3.


## Conclusión
Este experimento demuestra que los parámetros de generación en modelos fundacionales como IBM Granite tienen un impacto directo en la calidad del output. Ajustar adecuadamente valores como max_tokens y top_p permite:

- Lograr un equilibrio entre diversidad y precisión de las respuestas.

- Adaptar el modelo a tareas más estructuradas (resúmenes, clasificación) o creativas (generación libre).

- Personalizar el comportamiento del modelo sin necesidad de reentrenamiento.

Este tipo de afinamiento es esencial para integrar LLMs de manera eficiente en aplicaciones empresariales o asistencias inteligentes.